# Calculation of chemical weight<br/>
Lei Lei<br/>
A script uses the molmass package to calculate the weight of raw chemials required for synthesis.

In [1]:
from molmass import Formula

In [2]:
# Test the mass method of formula object
LSTO=Formula("La10Sr85Ti100O300", groups=None)
LSTO.mass/100

184.232762

In [3]:
# Test the composition method of formula object
TiO2=Formula("TiO2")
TiO2.composition(False)

(('O', 2, 31.99881, 0.4006571773328287), ('Ti', 1, 47.867, 0.5993428226671713))

Input data, the information of chemicals and target compouds.

In [4]:
raw_materials={
    "La": "La2O3",
    "Sr": "SrCO3",
    "Ti": "TiO2",
}

# Compounds and grams to synthesis
TargetCompounds={
    "La10Sr85Ti100O300": 5,
    "La20Sr70Ti100O300": 5,
    "La30Sr55Ti100O300": 5,
    "La40Sr40Ti100O300": 5,
}

Calculate the number of atoms and molar mass of the chemicals.

In [5]:
# Compositions & number
for element, chemical in raw_materials.items():
    chemical_formula=Formula(chemical)
    compositions=chemical_formula.composition(False)
    for composition in compositions:
        if composition[0]==element:
            multiplicity=composition[1]
            globals()[chemical]=[multiplicity, chemical_formula.mass]
    print(chemical +":", globals()[chemical])

La2O3: [2, 325.80915500000003]
SrCO3: [1, 147.62895500000002]
TiO2: [1, 79.86581]


Calculate the moles of elements, store the results into a dict.

In [6]:
for compound, weight in TargetCompounds.items():
    LSTO=Formula(compound)
    globals()[compound+"_moles"], globals()[compound+"_composition"]=weight/LSTO.mass, LSTO.composition(False)
    # print(globals()[compound+"_moles"], "\n", globals()[compound+"_composition"])
    globals()[compound] = {}
    for element in globals()[compound+"_composition"]:
        if any(element[0] in key for key in raw_materials):
            # The moles of elements
            globals()[compound][element[0]] = globals()[compound+"_moles"]*element[1]
    print(globals()[compound])

{'La': 0.0027139581178292275, 'Sr': 0.023068644001548434, 'Ti': 0.027139581178292273}
{'La': 0.0054059808063138225, 'Sr': 0.018920932822098377, 'Ti': 0.02702990403156911}
{'La': 0.008076332933063094, 'Sr': 0.014806610377282339, 'Ti': 0.02692110977687698}
{'La': 0.010725275118456237, 'Sr': 0.010725275118456237, 'Ti': 0.026813187796140597}


Define a function to calculate the weight of raw materials:
$$
m(\mathrm{chemical})=\frac{n(\mathrm{element}) \times M(\mathrm{chemical})}{N}
$$
where $M(\mathrm{chemical})$ and $N$ are the molar mass and multiplicity (<i>i.e.</i> the number of atoms in the chemical formula) of the element. 

In [7]:
def calc_weight(n, mm, multiple):
    return n * mm / multiple

Calculate the weight of chemicals to use by the `calc_weight` function.

In [8]:
for compound, weight in TargetCompounds.items():
    globals()[compound+"_chemicals"]={}
    for element, chemical in raw_materials.items():
        globals()[compound+"_chemicals"][chemical] = calc_weight (globals()[compound][element], globals()[chemical][1], globals()[chemical][0])
    print(globals()[compound+"_chemicals"])

{'La2O3': 0.4421162005376656, 'SrCO3': 3.405599807215614, 'TiO2': 2.167524633865067}
{'La2O3': 0.8806590192256627, 'SrCO3': 2.7932775401515846, 'TiO2': 2.1587651797035323}
{'La2O3': 1.3156716042099792, 'SrCO3': 2.1858844170903478, 'TiO2': 2.1500762384291994}
{'La2O3': 1.747196411743376, 'SrCO3': 1.5833611578251958, 'TiO2': 2.1414569620208836}


In [9]:
# Write a markdown file as record.
from time import gmtime, strftime
Title="# Calculation of chemical weights\n"
author="Lei Lei\n"
time = strftime("%Y-%m-%d %H:%M:%S\n", gmtime())
description="Data generated by the code in the jupyter notebook `calc_chemical_weight.ipynb`."
filename="AD-compositions.md"

with open (filename, 'w') as file:
    file.write(Title)
    file.write(author)
    file.write(time)
    file.write(description)
    for compound, weight in TargetCompounds.items():
        file.write("## " + compound + "\n")
        for key, value in globals()[compound+"_chemicals"].items():
            file.write("{0}: {1:.4f} g\n".format(key, value))
file.close

<function TextIOWrapper.close()>